In [1]:
# imports iniciales
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import utils
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

In [2]:
events = utils.get_events()
events.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,day,hora
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18,0
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18,0
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18,0
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18,0
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18,0


In [3]:
# Tomo como sesión media hora
events = events.sort_values(by='timestamp')
events['timestamp_anterior'] = events.groupby('person')['timestamp'].shift()
events['start_session'] = ((events['timestamp'] - events['timestamp_anterior']) >= timedelta(minutes=30)) | events['timestamp_anterior'].isnull()
events['start_session'] = events['start_session'].astype(int)
events['session_num'] = events.groupby('person')['start_session'].cumsum()
events['session_id'] =  events['person'] + '-' + events['session_num'].astype(str)

In [4]:
session_df = events.groupby(['session_id','person']).agg({'event':'count', 'timestamp':['min','max']})
session_df.columns = session_df.columns.droplevel()
session_df.columns = ['number_events','timestamp_min','timestamp_max']
session_df['total_time'] = ((session_df['timestamp_max']) - session_df['timestamp_min']).astype('timedelta64[s]')
session_df = session_df.reset_index().sort_values(by='session_id')
session_df.head()

,session_id,person,number_events,timestamp_min,timestamp_max,total_time
0,0008ed71-1,0008ed71,1,2018-05-17 12:27:47,2018-05-17 12:27:47,0.0
1,0008ed71-2,0008ed71,2,2018-05-17 13:44:59,2018-05-17 13:45:00,1.0
2,0008ed71-3,0008ed71,3,2018-05-17 16:21:54,2018-05-17 16:28:37,403.0
3,00091926-1,00091926,7,2018-05-03 22:08:29,2018-05-03 22:09:59,90.0
4,00091926-10,00091926,4,2018-05-11 02:23:38,2018-05-11 02:23:57,19.0


In [5]:
sessions_by_person = session_df.groupby(['person']).agg({'session_id':'count', 'total_time':['sum','mean']})
sessions_by_person.columns = sessions_by_person.columns.droplevel()
sessions_by_person.columns = ['sessions', 'total_time', 'mean_time_by_session']
sessions_by_person = sessions_by_person.reset_index()